In [67]:
import pandas as pd 
import numpy as np
import glob
import os
import time

In [ ]:

#모든 Tier의 MatchData 하나로 병합

# 1단계: 파일 찾기
files = glob.glob("TFT_*_MatchData.*")

print("찾은 파일:")
for f in files:
    print(f"  {f}")
print()


# 2단계: 파일 읽어서 합치기
all_data = []  

for file in files:
    df = pd.read_csv(file) 
    
    # 티어 이름 추출 (TFT_Challenger_MatchData.csv → Challenger)
    tier_name = file.split("_")[1]
    df["Tier"] = tier_name
    all_data.append(df)
    
    print(f"{tier_name}: {len(df):,}개")


# 3단계: 하나로 합치기
merged = pd.concat(all_data, ignore_index=True)
print(f"\n합친 결과: {len(merged):,}개")


# 4단계: 중복 제거
# gameID 컬럼이 있으면 중복 제거
if 'gameID' in merged.columns:
    merged = merged.drop_duplicates(subset=['gameID'])
    print(f"중복 제거 후: {len(merged):,}개")
else:
    print("중복 제거 안 함 (gameID 컬럼 없음)")

# 5단계: 저장하기
merged.to_csv("TFT_All_Tier.csv", index=False, encoding='utf-8-sig')
print("\n 완료! → TFT_All_Tier.csv 파일 생성됨")


# 6단계: 결과 확인
print("\n티어별 개수:")
print(merged['Tier'].value_counts())

print("\n데이터 미리보기:")
merged.head()

In [70]:
df = pd.read_csv("TFT_ALL_Tier.csv")

In [71]:

# 결측치 확인
df.isnull().sum()

gameId            0
gameDuration      0
level             0
lastRound         0
Ranked            0
ingameDuration    0
combination       0
champion          0
Tier              0
dtype: int64

In [72]:
print("Raw_data:", len(df), "개")
print()

Raw_data: 399998 개



In [73]:
# 게임당 8명이 아닌 게임 제거

game_counts = df.groupby('gameId').size()
outlier_games = game_counts[game_counts != 8].index

before = len(df)
df = df[~df['gameId'].isin(outlier_games)]
after = len(df)

print(f"8명 아닌 게임 제거: {before - after}개 제거됨 ({len(outlier_games)}개 게임)")

8명 아닌 게임 제거: 318개 제거됨 (21개 게임)


In [74]:
# 레벨 이상치 제거(1~9)
before = len(df)
df = df[(df['level'] >= 1) & (df['level'] <= 9)]
after = len (df)

print(f"레벨 이상치 제거: {before - after}개 제거됨")

레벨 이상치 제거: 0개 제거됨


In [75]:
# 게임 시간 이상치 제거 (10~60분)
outlier_duration = df[(df['gameDuration'] < 600) | (df['gameDuration'] > 3600)]

print(f"게임시간 이상치: {len(outlier_duration)}개")

# 이상치 확인하기
print(df[(df['gameDuration'] < 600) | (df['gameDuration'] > 3600)][['gameId', 'gameDuration']].head())
    
before = len(df)
df = df[(df['gameDuration'] >=600) & (df['gameDuration'] <= 3600)]
after = len(df)

print(f"게임시간 이상치 제거: {before - after}개 제거됨")



게임시간 이상치: 144개
              gameId  gameDuration
65959  KR_4347890020     38.888447
65960  KR_4347890020     38.888447
65961  KR_4347890020     38.888447
65962  KR_4347890020     38.888447
65963  KR_4347890020     38.888447
게임시간 이상치 제거: 144개 제거됨


In [76]:
# combination 빈 값 제거

before = len(df)
df = df[df['combination'].notnull()]
df = df[df['combination'].astype(str).str.len() > 0]
after = len(df)

print(f"comb 빈 값 제거: {before - after}개 제거됨")

comb 빈 값 제거: 0개 제거됨


In [77]:
# champion 빈 값 제거

before = len(df)
df = df[df['champion'].notnull()]
after = len(df)

print(f"champion 빈 값 제거: {before - after}개 제거됨")

champion 빈 값 제거: 0개 제거됨


In [78]:
# 이상치 처리 후, 저장

df.to_csv("TFT_Clean_Data.csv", index=False, encoding='utf-8-sig')
print("저장 완료: TFT_Clean_Data.csv")

저장 완료: TFT_Clean_Data.csv
